# GeneRIF

Created by: Charles Dai <br>
Credit to: Moshe Silverstein

Data Source Home: https://www.ncbi.nlm.nih.gov/gene/about-generif/ <br>
Data Source Download: https://ftp.ncbi.nih.gov/gene/GeneRIF/

In [ ]:
# appyter init
from appyter import magic
magic.init(lambda _=globals: _())

In [ ]:
import sys
import os
from datetime import date

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import harmonizome.utility_functions as uf
import harmonizome.lookup as lookup

In [ ]:
# from clustergrammer_widget import *
# net = Network(clustergrammer_widget)

In [ ]:
%load_ext autoreload
%autoreload 2

### Notebook Information

In [ ]:
print('This notebook was run on:', date.today(), '\nPython version:', sys.version)

# Initialization

### Load Mapping Dictionaries

In [ ]:
symbol_lookup, geneid_lookup = lookup.get_lookups()

### Output Path

In [ ]:
output_name = 'generif'

path = 'Output/GeneRIF'
if not os.path.exists(path):
    os.makedirs(path)

In [ ]:
%%appyter hide_code
{% do SectionField(
    name='data',
    title='Load Data',
    subtitle='Upload Files from the GeneRIF Dataset',
) %}

# Load Data

In [ ]:
%%appyter code_exec

df = pd.read_csv({{FileField(
    constraint='.*\.gz$',
    name='interactions', 
    label='Interactions (gz)', 
    default='Input/GeneRIF/interactions.gz',
    section='data')
}}, sep='\t', usecols=['#tax_id', 'gene_id', 'keyphrase', 'name.1'])

In [ ]:
df.head()

In [ ]:
df.shape

# Pre-process Data

## Get Relevant Data

In [ ]:
# Only relevant species
df = df[np.logical_or.reduce([
    df['#tax_id'] == 9606, # human
    df['#tax_id'] == 10090, # mouse
    df['#tax_id'] == 10116 # rat
])]
# Only entries that have keyphrases
df = df[df['keyphrase'] != '-']
df.shape

In [ ]:
# Concatenate full attribute
df['Effect'] = df['keyphrase'] + ' ' + df['name.1']
df = df[['gene_id', 'Effect']].set_index('gene_id')
df.head()

## Map Gene ID to Symbol

In [ ]:
id_to_symbol = {v: k for k, v in geneid_lookup.items()}
df.index = df.index.map(id_to_symbol)
df.index.name = 'Gene Symbol'
df.head()

# Filter Data

## Map Gene Symbols to Up-to-date Approved Gene Symbols

In [ ]:
df = uf.map_symbols(df, symbol_lookup, remove_duplicates=True)
df.shape

# Analyze Data

## Create Binary Matrix

In [ ]:
binary_matrix = uf.binary_matrix(df)
binary_matrix.head()

In [ ]:
binary_matrix.shape

In [ ]:
uf.save_data(binary_matrix, path, output_name + '_binary_matrix', 
            compression='npz', dtype=np.uint8)

## Create Gene List

In [ ]:
gene_list = uf.gene_list(binary_matrix, geneid_lookup)
gene_list.head()

In [ ]:
gene_list.shape

In [ ]:
uf.save_data(gene_list, path, output_name + '_gene_list',
            ext='tsv', compression='gzip', index=False)

## Create Attribute List

In [ ]:
attribute_list = uf.attribute_list(binary_matrix)
attribute_list.head()

In [ ]:
attribute_list.shape

In [ ]:
uf.save_data(attribute_list, path, output_name + '_attribute_list',
            ext='tsv', compression='gzip')

## Create Gene and Attribute Set Libraries

In [ ]:
uf.save_setlib(binary_matrix, 'gene', 'up', path, output_name + '_gene_up_set')

In [ ]:
uf.save_setlib(binary_matrix, 'attribute', 'up', path, 
                           output_name + '_attribute_up_set')

## Create Attribute Similarity Matrix

In [ ]:
attribute_similarity_matrix = uf.similarity_matrix(binary_matrix.T, 'jaccard', sparse=True)
attribute_similarity_matrix.head()

In [ ]:
uf.save_data(attribute_similarity_matrix, path,
            output_name + '_attribute_similarity_matrix', 
            compression='npz', symmetric=True, dtype=np.float32)

In [ ]:
# net.load_df(attribute_similarity_matrix.iloc[:,:].copy())
# net.filter_N_top('row', rank_type='sum', N_top=300)
# net.cluster()
# net.widget()

## Create Gene Similarity Matrix

In [ ]:
gene_similarity_matrix = uf.similarity_matrix(binary_matrix, 'jaccard', sparse=True)
gene_similarity_matrix.head()

In [ ]:
uf.save_data(gene_similarity_matrix, path, 
            output_name + '_gene_similarity_matrix',
            compression='npz', symmetric=True, dtype=np.float32)

## Create Gene-Attribute Edge List

In [ ]:
edge_list = uf.edge_list(binary_matrix)
uf.save_data(edge_list, path, output_name + '_edge_list', 
        ext='tsv', compression='gzip')

# Create Downloadable Save File

In [ ]:
uf.archive(path)

### Link to download output files: [click here](./output_archive.zip)